In [91]:
import nltk

In [92]:
import pandas as pd
import numpy as np

In [93]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

In [94]:
from collections import Counter

In [95]:
from imblearn.under_sampling import RandomUnderSampler

# NLP Pipeline 

![](images/pipeline-walkthrough1.png)

Below is a to do list when converting text into vector form: 

**Clean text and Create a Bag of Words (BoW)**
>1. Lowercase the text
2. Tokenize 
3. Strip out punctuation or undesirable text
4. Remove Stopwords 
5. Stemming or Lemmatizing
6. Compute N-Grams
7. Use this to create BoW

**Vectorize BoW**
>8. Term Frequencies
9. Document Frequencies
10. TF-IDF
11. Normalize vectors

Let's go through both what each of these steps are and how to do them in python with the following corpus of comments about data science...
 

In [96]:
#funtion to cut off extra intro paragraphs from beautiful soup scrape

# def trim_fat(string):
#     return string[35:-115]

In [97]:
# function to vectorize the type_of_material series into a y target vector.
def vectorize_type(ser):
    y = ser.copy()
    y.replace({'Op-Ed': 1,'News': 0}, inplace=True)
    return y

In [98]:
#rate of correct predictions out of total predictions
def metrics_(tn, fp, fn, tp):
    accuracy = (tp + tn) / (tn + fn + tp + fp)
    print(f'accuracy = {accuracy}')
    recall = (tp) / (tp + fn)
    print(f'recall = {recall}')
    precision = (tp) / (tp + fp)
    print(f'precision = {precision}')

In [99]:
%%time
_2019 = pd.read_csv('data/trim2019_text_type.csv', index_col='Unnamed: 0')

CPU times: user 1.65 s, sys: 311 ms, total: 1.96 s
Wall time: 1.98 s


In [100]:
%%time
_2019_df = _2019.copy()

CPU times: user 798 µs, sys: 16 µs, total: 814 µs
Wall time: 852 µs


In [101]:
#define X. X is currently pandas series of unsplit strings

X = _2019_df.text

In [102]:
type(X[0])

str

In [103]:
type(X)

pandas.core.series.Series

In [104]:
# define y as a series of op-ed or news

y = _2019_df.type_of_material

In [105]:
%%time
# vectorize y in to (1, 0) (op-ed, news)

y = vectorize_type(y)

CPU times: user 10.3 ms, sys: 6.08 ms, total: 16.4 ms
Wall time: 15.7 ms


In [106]:
#turn series into list...

corpus = list(X)

# sklearn TfidfVectorizer(stop_words='english', strip_accents='ascii')

In [107]:
# %%time
# #create vectorizer

# vectorizer = TfidfVectorizer(#input='content', 
# #                 encoding='utf-8', 
# #                 decode_error='strict', 
#                  strip_accents=None, 
#                  lowercase=True, 
# #                 preprocessor=None, 
# #                 tokenizer=None, 
# #                 analyzer='word', 
#                  stop_words='english', 
# #                 token_pattern='(?u)\b\w\w+\b', 
# #                 ngram_range=(1, 1), 
# #                 max_df=1.0, 
# #                 min_df=1, 
#                  max_features=None, 
# #                 vocabulary=None, 
# #                 binary=False, 
# #                 dtype=<class 'numpy.float64'>, 
# #                 norm='l2', 
# #                 use_idf=True, 
# #                 smooth_idf=True, 
# #                 sublinear_tf=False
# )
# X = vectorizer.fit_transform(corpus)

In [108]:
# %%time
# feature_names = vectorizer.get_feature_names()

In [109]:
# %%time
# stop_words = vectorizer.get_stop_words()

In [110]:
# feature_names;

In [111]:
# len(feature_names)

In [112]:
# len(stop_words)

# resample class size w/ imbalanced learn

In [113]:
# y.shape

In [114]:
# X.shape

In [115]:
# %%time
# #balance the classes

# from imblearn.under_sampling import RandomUnderSampler
# rus = RandomUnderSampler(random_state=0)

In [116]:
# %%time
# #X, y --> X_resampled, y_resampled
# X_resampled, y_resampled = rus.fit_resample(X, y)


# #return a list of tuples for item, and count of item. in this case 4139 each
# print(sorted(Counter(y_resampled).items()))

In [117]:
# y_resampled.shape

In [118]:
# X_resampled.shape

In [119]:
# %%time
# #test, train, split

# X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, random_state=50)

# Multinomial Naive Bayes/imbalanced learn/TFIDF vectorizer

In [120]:
# %%time

# multinm_clf = MultinomialNB()
# multinm_clf.fit(X_train, y_train)

In [121]:
#multinm_clf.class_count_

In [122]:
#y_pred = multinm_clf.predict(X_test)

In [123]:
# #clf.score = accuracy = 'true'(pos/neg) / total

# multinm_clf.score(X_test, y_test)

In [124]:
#confusion_matrix(y_true = y_test, y_pred = y_pred)

In [125]:
# tn, fp, fn, tp = confusion_matrix(y_true = y_test, y_pred = y_pred).ravel()
# (tn, fp, fn, tp)

In [126]:
# #rate of correct predictions out of total predictions

# accuracy = (tp + tn) / (tn + fn + tp + fp)
# accuracy

In [127]:
# # rate of actual op-ed articles, out of all the actual od-ed articles

# recall = (tp) / (tp + fn)
# recall

In [128]:
# # rate of correct predictions of op-ed articles out of all predictions

# precision = (tp) / (tp + fp)
# precision

In [129]:
# #rate of correct predictions out of total predictions

# accuracy = (tp + tn) / (tn + fn + tp + fp)
# print(f'accuracy = {accuracy}')
# recall = (tp) / (tp + fn)
# print(f'recall = {recall}')
# precision = (tp) / (tp + fp)
# print(f'precision = {precision}')

# random forest classifier

In [130]:
# class sklearn.ensemble.RandomForestClassifier(n_estimators=100, 
#                                               *, 
#                                               criterion='gini', 
#                                               max_depth=None, 
#                                               min_samples_split=2, 
#                                               min_samples_leaf=1, 
#                                               min_weight_fraction_leaf=0.0, 
#                                               max_features='auto', 
#                                               max_leaf_nodes=None, 
#                                               min_impurity_decrease=0.0, 
#                                               min_impurity_split=None, 
#                                               bootstrap=True, 
#                                               oob_score=False, 
#                                               n_jobs=None, 
#                                               random_state=None, 
#                                               verbose=0, 
#                                               warm_start=False, 
#                                               class_weight=None, 
#                                               ccp_alpha=0.0, 
#                                               max_samples=None)

In [131]:
# %%time

# rf_clf = RandomForestClassifier(max_depth=2, random_state=0)

In [132]:
# %%time

# rf_clf.fit(X_train, y_train)

In [133]:
# y_pred = rf_clf.predict(X_test)

In [134]:
# #clf.score = accuracy = 'true'(pos/neg) / total

# rf_clf.score(X_test, y_test)

In [135]:
# confusion_matrix(y_true = y_test, y_pred = y_pred)

In [136]:
# tn, fp, fn, tp = confusion_matrix(y_true = y_test, y_pred = y_pred).ravel()
# (tn, fp, fn, tp)

In [137]:
# #rate of correct predictions out of total predictions

# accuracy = (tp + tn) / (tn + fn + tp + fp)
# accuracy

In [138]:
# # rate of actual op-ed articles, out of all the actual od-ed articles

# recall = (tp) / (tp + fn)
# recall

In [139]:
# # rate of correct predictions of op-ed articles out of all predictions

# precision = (tp) / (tp + fp)
# precision

In [140]:
# #rate of correct predictions out of total predictions

# accuracy = (tp + tn) / (tn + fn + tp + fp)
# print(f'accuracy = {accuracy}')
# recall = (tp) / (tp + fn)
# print(f'recall = {recall}')
# precision = (tp) / (tp + fp)
# print(f'precision = {precision}')

In [141]:
# class sklearn.ensemble.RandomForestClassifier(n_estimators=100, 
#                                               *, 
#                                               criterion='gini', 
#                                               max_depth=None, 
#                                               min_samples_split=2, 
#                                               min_samples_leaf=1, 
#                                               min_weight_fraction_leaf=0.0, 
#                                               max_features='auto', 
#                                               max_leaf_nodes=None, 
#                                               min_impurity_decrease=0.0, 
#                                               min_impurity_split=None, 
#                                               bootstrap=True, 
#                                               oob_score=False, 
#                                               n_jobs=None, 
#                                               random_state=None, 
#                                               verbose=0, 
#                                               warm_start=False, 
#                                               class_weight=None, 
#                                               ccp_alpha=0.0, 
#                                               max_samples=None)

# NLTK 

In [142]:
nltk_2019_df = _2019.copy()

In [143]:
#define X. X is currently pandas series of unsplit strings

X_nltk = nltk_2019_df.text

In [144]:
# define y as a series of op-ed or news

y_nktk = nltk_2019_df.type_of_material

In [145]:
%%time
# vectorize y in to (1, 0) (op-ed, news)

y_nltk = vectorize_type(y_nktk)

CPU times: user 11.3 ms, sys: 1.43 ms, total: 12.7 ms
Wall time: 11.5 ms


In [146]:
#turn series into list...

corpus_nltk = list(X_nltk)

### tokenize w/ nltk

In [147]:
# %%time
# from nltk.tokenize import RegexpTokenizer
# regex_tokenizer = nltk.RegexpTokenizer(r"\w+")
# tokenized_punc = [regex_tokenizer.tokenize(article.lower())for article in corpus_nltk]

In [148]:
# (len(tokenized_punc[0])) #2218

In [149]:
# len(tokenized_punc)

In [150]:
# from nltk.tokenize import word_tokenize

In [151]:
# type(word_tokenize)

In [152]:
# %%time
# tokenized = [word_tokenize(article.lower()) for article in corpus_nltk]

# CPU times: user 4min 46s, sys: 2.39 s, total: 4min 48s
# Wall time: 4min 50s

In [153]:
# #list of list of strings. one list of strings per documents. list are various lengths around 1000

# len(tokenized[0]) #2596

In [154]:
from nltk.corpus import stopwords

## take out stop work via ntlk. does this work against sklearn when i vectorize

In [155]:
%%time
stop = set(stopwords.words('english'))
tokenized_docs = [[word for word in words if word not in stop]
            for words in tokenized_punc]

NameError: name 'tokenized_punc' is not defined

In [156]:
# #hopefully this reduced the number of strings / list

# len(tokenized[1])

In [157]:
# #docs is new tokenized, but with stop words removed

# len(tokenized_docs)

# stemming/lemmatization

In [158]:
# from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
# from nltk.stem.wordnet import WordNetLemmatizer

In [159]:
# porter = PorterStemmer()
snowball = SnowballStemmer('english')
# wordnet = WordNetLemmatizer()

In [160]:
# %%time
# docs_porter = [[porter.stem(word) for word in words]
#                    for words in docs]
# docs_snowball = [[snowball.stem(word) for word in words]
#                      for words in docs]
# docs_wordnet = [[wordnet.lemmatize(word) for word in words]
#                     for words in docs]

# CPU times: user 14min 59s, sys: 18.4 s, total: 15min 18s
# Wall time: 15min 26s

In [161]:
# %%time
# docs_porter = [[porter.stem(word) for word in words]
#                    for words in docs]

# CPU times: user 7min 16s, sys: 5.21 s, total: 7min 21s
# Wall time: 7min 22s

In [162]:
# %%time
# snowball_stemm = [[snowball.stem(word) for word in words]
#                      for words in tokenized_docs]

# # CPU times: user 5min 5s, sys: 5.98 s, total: 5min 11s
# # Wall time: 5min 13s

In [163]:
# %%time
# docs_wordnet = [[wordnet.lemmatize(word) for word in words]
#                     for words in docs]

# CPU times: user 1min 24s, sys: 4.5 s, total: 1min 28s
# Wall time: 1min 30s

In [164]:
# %%time
# ## Print the stemmed and lemmatized words from the first document
# print("%16s %16s %16s %16s" % ("word", "porter", "snowball", "lemmatizer"))
# for i in range(min(len(docs_porter[0]), len(docs_snowball[0]), len(docs_wordnet[0]))):
#     p, s, w = docs_porter[0][i], docs_snowball[0][i], docs_wordnet[0][i]
#     if len(set((p, s, w))) != 1:
#         print("%16s %16s %16s %16s" % (docs[0][i], p, s, w))
#         print(docs[0][i], w)


In [165]:
#docs and lemmatizer are the same?

# I choose SNOWBALL!!!! to sklearn

In [166]:
type(corpus_nltk)

list

In [167]:
type(corpus_nltk[0])

str

In [168]:
# snowball = SnowballStemmer('english')
# snowball_tokenized = [snowball.stem(word) for word in word_tokenize(doc.lower())]

In [169]:
%%time
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize

def snowball_tokenize(doc):
    snowball = SnowballStemmer('english')
    return [snowball.stem(word) for word in word_tokenize(doc.lower())]

CPU times: user 11 µs, sys: 0 ns, total: 11 µs
Wall time: 14.1 µs


In [170]:
%%time
#create vectorizer

vectorizer = TfidfVectorizer(#input='content', 
#                 encoding='utf-8', 
#                 decode_error='strict', 
                 strip_accents='ascii', 
                 lowercase=True, 
#                 preprocessor=None, 
                 tokenizer=snowball_tokenize, 
#                 analyzer='word', 
                 stop_words='english', 
#                 token_pattern='(?u)\b\w\w+\b', 
#                 ngram_range=(1, 1), 
#                 max_df=1.0, 
#                 min_df=1, 
                 max_features=None, 
#                 vocabulary=None, 
#                 binary=False, 
#                 dtype=<class 'numpy.float64'>, 
#                 norm='l2', 
#                 use_idf=True, 
#                 smooth_idf=True, 
#                 sublinear_tf=False
)



CPU times: user 38 µs, sys: 30 µs, total: 68 µs
Wall time: 70.8 µs


In [171]:
%%time
X_snowball = vectorizer.fit(corpus_nltk)

CPU times: user 13min 5s, sys: 7.11 s, total: 13min 12s
Wall time: 13min 16s


In [172]:
%%time
X_snowball = X_snowball.transform(corpus_nltk)

CPU times: user 13min 15s, sys: 7.39 s, total: 13min 23s
Wall time: 13min 29s


In [173]:
#class sklearn.feature_extraction.text.CountVectorizer(*, 
#                                                       input='content', 
#                                                       encoding='utf-8', 
#                                                       decode_error='strict', 
#                                                       strip_accents=None, 
#                                                       lowercase=True, 
#                                                       preprocessor=None, 
#                                                       tokenizer=None, 
#                                                       stop_words=None, 
#                                                       token_pattern='(?u)\b\w\w+\b', 
#                                                       ngram_range=(1, 1), 
#                                                       analyzer='word', 
#                                                       max_df=1.0, 
#                                                       min_df=1, 
#                                                       max_features=None, 
#                                                       vocabulary=None, 
#                                                       binary=False, 
#                                                       dtype=<class 'numpy.int64'>
#)

In [174]:
count_vectorizer = CountVectorizer(strip_accents='None',
                                   lowercase=True,
                                   tokenizer=snowball_tokenize,
                                   stop_words='english',
                                   max_features=None)

In [175]:
%%time
#balance the classes

from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=0)

CPU times: user 22 µs, sys: 0 ns, total: 22 µs
Wall time: 25 µs


In [176]:
%%time
#X, y --> X_resampled, y_resampled
X_resampled, y_resampled = rus.fit_resample(X_snowball, y_nltk)
print(sorted(Counter(y_resampled).items()))

[(0, 4139), (1, 4139)]
CPU times: user 25.3 ms, sys: 8.59 ms, total: 33.9 ms
Wall time: 32.7 ms


In [177]:
%%time
#test, train, split

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, random_state=50)

CPU times: user 11.5 ms, sys: 8.72 ms, total: 20.2 ms
Wall time: 18.9 ms


In [178]:
%%time

rf_clf = RandomForestClassifier(max_depth=2, random_state=0)

CPU times: user 40 µs, sys: 0 ns, total: 40 µs
Wall time: 42 µs


In [179]:
%%time

rf_clf.fit(X_train, y_train)

CPU times: user 397 ms, sys: 28.7 ms, total: 425 ms
Wall time: 427 ms


RandomForestClassifier(max_depth=2, random_state=0)

In [180]:
y_pred = rf_clf.predict(X_test)

In [181]:
#clf.score = accuracy = 'true'(pos/neg) / total

rf_clf.score(X_test, y_test)

0.9304347826086956

In [194]:
tn, fp, fn, tp = confusion_matrix(y_true = y_test, y_pred = y_pred).ravel()
(tn, fp, fn, tp)
metrics_(tn, fp, fn, tp)
print(f'tn={tn}, fp={fp}, fn={fn}, tp={tp})')
print(rf_clf.n_features_)
print(rf_clf.n_classes_)
print(rf_clf.n_outputs_)
# what are the actual parametrs set in the function. something specific about the random forest i forgot

accuracy = 0.9304347826086956
recall = 0.8942857142857142
precision = 0.9660493827160493
tn=987, fp=33, fn=111, tp=939)
263494
2
1


In [183]:
nltk_features = vectorizer.get_feature_names()

In [184]:
nltk_stop = vectorizer.get_stop_words()

In [185]:
type(nltk_features)

list

In [186]:
len(nltk_stop)

318

In [187]:
X_snowball.shape

(41748, 263494)

In [188]:
# visualizing the bag of words
columns = nltk_features

In [189]:
feature_import = rf_clf.feature_importances_

In [190]:
# property feature_importances_
# The impurity-based feature importances.

# The higher, the more important the feature. The importance of a feature is computed as the (normalized) total reduction of the criterion brought by that feature. It is also known as the Gini importance.

# Warning: impurity-based feature importances can be misleading for high cardinality features (many unique values). See sklearn.inspection.permutation_importance as an alternative.

# Returns
# feature_importances_ndarray of shape (n_features,)
# The values of this array sum to 1, unless all trees are single node trees consisting of only the root node, in which case it will be an array of zeros.



In [208]:
# feat_scores = pd.Series(feature_import,
#                            index=nltk_features)
# feat_scores = feat_scores.sort_values()
# ax = feat_scores.plot(kind='barh', figsize=(10,20))
# ax.set_title('"Decrease in Impurity" Importance')
# ax.set_xlabel('Average contribution to the reduction in variance');

In [209]:
# r = permutation_importance(rf_clf, X_test, y_test, n_repeats=30, random_state=0)
# for index in r.importances_mean.argsort()[::-1]:
#     if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
#         print(f"{feature_names[i]:<8}"
#               f"{r.importances_mean[i]:.3f}"
#               f" +/- {r.importances_std[i]:.3f}")

In [200]:
%%time
#create vectorizer

test_vectorizer = TfidfVectorizer(#input='content', 
#                 encoding='utf-8', 
#                 decode_error='strict', 
                 strip_accents='ascii', 
                 lowercase=True, 
#                 preprocessor=None, 
                 tokenizer=snowball_tokenize, 
#                 analyzer='word', 
                 stop_words='english', 
#                 token_pattern='(?u)\b\w\w+\b', 
#                 ngram_range=(1, 1), 
#                 max_df=1.0, 
#                 min_df=1, 
                 max_features=450, 
#                 vocabulary=None, 
#                 binary=False, 
#                 dtype=<class 'numpy.float64'>, 
#                 norm='l2', 
#                 use_idf=True, 
#                 smooth_idf=True, 
#                 sublinear_tf=False
)


CPU times: user 14.6 ms, sys: 12.9 ms, total: 27.4 ms
Wall time: 27.9 ms


In [201]:
%%time
test_x = test_vectorizer.fit_transform(corpus_nltk)

CPU times: user 12min 55s, sys: 3.9 s, total: 12min 59s
Wall time: 13min 4s


In [235]:
type(test_x)

scipy.sparse.csr.csr_matrix

In [236]:
%%time
#balance the classes

from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=0)

#X, y --> X_resampled, y_resampled
X_resampled, y_resampled = rus.fit_resample(test_x, y_nltk)
print(sorted(Counter(y_resampled).items()))

[(0, 4139), (1, 4139)]
CPU times: user 16.3 ms, sys: 9.15 ms, total: 25.4 ms
Wall time: 24.3 ms


In [237]:
%%time
#test, train, split

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, random_state=50)

CPU times: user 6.13 ms, sys: 4.64 ms, total: 10.8 ms
Wall time: 9.45 ms


In [238]:
%%time
test450_rf_clf = RandomForestClassifier(max_depth=2, random_state=0)
test450_rf_clf.fit(X_train, y_train)
test450_y_pred = test450_rf_clf.predict(X_test)

CPU times: user 645 ms, sys: 14.2 ms, total: 659 ms
Wall time: 662 ms


In [239]:
%%time
tn, fp, fn, tp = confusion_matrix(y_true = y_test, y_pred = y_pred).ravel()
(tn, fp, fn, tp)
metrics_(tn, fp, fn, tp)
print(f'tn={tn}, fp={fp}, fn={fn}, tp={tp})')
print(rf_clf.n_features_)
print(rf_clf.n_classes_)
print(rf_clf.n_outputs_)
test450_feat_names = test_vectorizer.get_feature_names()
print(type(test450_feat_names))
feature_import = test450_rf_clf.feature_importances_
print(type(feature_import))
feature_import.shape
# what are the actual parametrs set in the function. something specific about the random forest i forgot

accuracy = 0.9304347826086956
recall = 0.8942857142857142
precision = 0.9660493827160493
tn=987, fp=33, fn=111, tp=939)
263494
2
1
<class 'list'>
<class 'numpy.ndarray'>
CPU times: user 13.5 ms, sys: 2.98 ms, total: 16.5 ms
Wall time: 14.8 ms


(450,)

In [247]:
type(test450_feat_names)

list

In [248]:
type(feature_import)

numpy.ndarray

In [250]:
X_test.shape

(2070, 450)

In [251]:
y_test.shape

(2070,)

In [249]:
# feat_scores = pd.Series(feature_import,
#                            index=test450_feat_names)
# feat_scores = feat_scores.sort_values()
# ax = feat_scores.plot(kind='barh', figsize=(10,20))
# ax.set_title('"Decrease in Impurity" Importance')
# ax.set_xlabel('Average contribution to the reduction in variance');

In [255]:
%%time
r = permutation_importance(test450_rf_clf, X_test.toarray(), y_test, n_repeats=30, random_state=0)
for i in r.importances_mean.argsort()[::-1]:
    if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
        print(f"{test450_feat_names[i]:<8}"
              f"{r.importances_mean[i]:.3f}"
              f" +/- {r.importances_std[i]:.3f}")

@       0.022 +/- 0.003
facebook0.022 +/- 0.003
twitter 0.006 +/- 0.001
york    0.006 +/- 0.002
)       0.005 +/- 0.002
follow  0.004 +/- 0.001
said    0.003 +/- 0.001
polici  0.001 +/- 0.000
data    0.000 +/- 0.000
econom  0.000 +/- 0.000
tuesday 0.000 +/- 0.000
elect   0.000 +/- 0.000
mean    0.000 +/- 0.000
CPU times: user 4min 2s, sys: 3.99 s, total: 4min 6s
Wall time: 4min 8s


In [256]:
from sklearn.inspection import permutation_importance